In [99]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle 


In [100]:
# load the data 
df=pd.read_csv('Churn_Modelling.csv')
df.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [101]:
## Preprocess the data 
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [102]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [103]:
df.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [104]:
#Apply encoding for geography and gender as they are categorical features 
label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [105]:
# Cannot apply label encoding to geography column and apply onehot encoder 
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder=OneHotEncoder()
geo_encoder=one_hot_encoder.fit_transform(df[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [106]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [107]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [108]:
geo_encoder_df=pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder.get_feature_names_out(['Geography']))

In [109]:
#COmbine the one hot encoded columns with the original data 
data=pd.concat([df.drop(['Geography'],axis=1),geo_encoder_df], axis=1)
data.head()


# Categorical columns converted to numerical columns 

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [110]:
# Save the encoders and scalers 
with open('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder,file)
    
with open('one_hot_encoder_geo.pkl','wb')as file:
    pickle.dump(one_hot_encoder,file)

In [111]:
# Divide the columns into training and testing set 
X=data.drop('Exited',axis=1)
y=data['Exited']

#Split the data into training and testing data 
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

## Do the scaling of data 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [112]:
X_train.shape,X_test.shape

((8000, 12), (2000, 12))

In [113]:
y_train.shape,y_test.shape

((8000,), (2000,))

In [114]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [115]:
with open('scaler.pkl','wb') as file : 
    pickle.dump(scaler,file)

## ANN implementation 

We will have sequential model for neural networks 

In [116]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

In [117]:
X_train.shape[1]

12

In [118]:
## Build ANN model 
# Step 1 - initialize sequential model 
#input_shape has to be written for the first hidden layer only bcoz it is getting connected to the inputs 
# no need to write input_shape from  the second hidden layer 
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  #using 12 inputs in the hidden layer 1 connected to the input layer and 
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') #output layer 
])

#64,32,1  are the number of hidden  neurons taken in the hidden layer 

c:\Users\vpoor\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [119]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [120]:
import tensorflow as tf 
optm=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x000001AA27A22F80>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [121]:
# Compile the model 
model.compile(optimizer=optm, loss="binary_crossentropy",metrics=['accuracy'])

In [122]:
## Setup the tensorboard 
import tensorflow
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = f"log/fit/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
tensorflow_call_backs=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [123]:
# Setup early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
#patience=5 means be patient for atleast 5 epochs 
#restore_best_weights means that out of all the epochs which one carries the best weight 

In [124]:
# Train the model 
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,
                  callbacks=[tensorflow_call_backs,early_stopping_callback])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8185 - loss: 0.4243 - val_accuracy: 0.8600 - val_loss: 0.3522
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8603 - loss: 0.3552 - val_accuracy: 0.8440 - val_loss: 0.3542
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8580 - loss: 0.3507 - val_accuracy: 0.8575 - val_loss: 0.3479
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8611 - loss: 0.3433 - val_accuracy: 0.8570 - val_loss: 0.3508
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8625 - loss: 0.3373 - val_accuracy: 0.8540 - val_loss: 0.3602
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8669 - loss: 0.3220 - val_accuracy: 0.8535 - val_loss: 0.3500
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8649 - loss: 0.3298 - val_accuracy: 0.8600 - val_loss: 0.3435
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8647 - loss: 0.3315 - val_accu

In [125]:
model.save('model.h5')

In [126]:
#Load tensorboard extension 
%load_ext tensorboard 
#%reload_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [127]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6006 (pid 64012), started 2:37:36 ago. (Use '!kill 64012' to kill it.)

In [128]:
## Load the pickle file 

In [129]:
df.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')